<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1">Table of Contents</a></span></li><li><span><a href="#Preprocessing---The-Movie-Dataset" data-toc-modified-id="Preprocessing---The-Movie-Dataset-2">Preprocessing - The Movie Dataset</a></span><ul class="toc-item"><li><span><a href="#Load-original-&quot;the-movie-dataset&quot;" data-toc-modified-id="Load-original-&quot;the-movie-dataset&quot;-2.1">Load original "the-movie-dataset"</a></span></li><li><span><a href="#Removing-uninformative-features" data-toc-modified-id="Removing-uninformative-features-2.2">Removing uninformative features</a></span><ul class="toc-item"><li><span><a href="#Removing-adult" data-toc-modified-id="Removing-adult-2.2.1">Removing adult</a></span></li><li><span><a href="#Removing-budget" data-toc-modified-id="Removing-budget-2.2.2">Removing budget</a></span></li><li><span><a href="#Removing-revenue" data-toc-modified-id="Removing-revenue-2.2.3">Removing revenue</a></span></li></ul></li><li><span><a href="#Removing-Rows---Removing-movies-out-of-the-dataset" data-toc-modified-id="Removing-Rows---Removing-movies-out-of-the-dataset-2.3">Removing Rows - Removing movies out of the dataset</a></span><ul class="toc-item"><li><span><a href="#Removing-broken-data-points" data-toc-modified-id="Removing-broken-data-points-2.3.1">Removing broken data points</a></span></li><li><span><a href="#Removing-non-existent-or-not-yet-released-movies" data-toc-modified-id="Removing-non-existent-or-not-yet-released-movies-2.3.2">Removing non existent or not yet released movies</a></span></li></ul></li><li><span><a href="#Transforming-features" data-toc-modified-id="Transforming-features-2.4">Transforming features</a></span><ul class="toc-item"><li><span><a href="#Transforming-genres-to-one-hot" data-toc-modified-id="Transforming-genres-to-one-hot-2.4.1">Transforming genres to one hot</a></span></li><li><span><a href="#Transforming-original_language-to-one-hot-(only-languages-which-are-very-frequent)" data-toc-modified-id="Transforming-original_language-to-one-hot-(only-languages-which-are-very-frequent)-2.4.2">Transforming original_language to one hot (only languages which are very frequent)</a></span></li><li><span><a href="#Transforming-release_date-to-release_year" data-toc-modified-id="Transforming-release_date-to-release_year-2.4.3">Transforming release_date to release_year</a></span></li></ul></li><li><span><a href="#Adding-new-features-from-credits.csv" data-toc-modified-id="Adding-new-features-from-credits.csv-2.5">Adding new features from credits.csv</a></span><ul class="toc-item"><li><span><a href="#Adding-features:-star_1,-star_2,-star_3,-director,-writer,-producer" data-toc-modified-id="Adding-features:-star_1,-star_2,-star_3,-director,-writer,-producer-2.5.1">Adding features: star_1, star_2, star_3, director, writer, producer</a></span></li></ul></li><li><span><a href="#Adding-most-frequent-keywords-from-keywords.csv" data-toc-modified-id="Adding-most-frequent-keywords-from-keywords.csv-2.6">Adding most frequent keywords from keywords.csv</a></span></li><li><span><a href="#Exporting-the-preprocessed-&quot;The-Movie-Dataset&quot;-to-csv" data-toc-modified-id="Exporting-the-preprocessed-&quot;The-Movie-Dataset&quot;-to-csv-2.7">Exporting the preprocessed "The Movie Dataset" to csv</a></span></li></ul></li></ul></div>

# Preprocessing - The Movie Dataset

In [1]:
import os
import json
import numpy as np
import pandas as pd
from ast import literal_eval

## Load original "the-movie-dataset"

In [2]:
data_path = os.path.join(os.getcwd(), 'data', 'the-movies-dataset')
movies = pd.read_csv(os.path.join(data_path, 'movies_metadata.csv'))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

## Removing uninformative features

In [4]:
del movies['belongs_to_collection'] # boring
del movies['homepage'] # boring
#del movies['imdb_id'] # needed for crawling the covers?
#del movies['poster_path'] # needed for crawling the covers?
del movies['original_title'] # boring; title is sufficient
del movies['production_companies'] # boring 
del movies['production_countries'] # boring
del movies['video'] # boring

### Removing adult

In [5]:
# checking missing values
feature = 'adult'
print("Feature: {}, Missing Values: {}, Data Percentage: {:.2f}\n".format(
    feature, len(movies[feature])-len(movies[feature].dropna()), len(movies[feature].dropna())/len(movies)))

# check for value diversity
print('Amount of adult movies: {}\nAmount of non-adult movies: {}'.format(len(movies[movies['adult'] == 'True']), len(movies[movies['adult'] == 'False'])))

Feature: adult, Missing Values: 0, Data Percentage: 1.00

Amount of adult movies: 9
Amount of non-adult movies: 45454


In [6]:
movies[movies['adult'] == 'True'][['adult', 'title']]

,adult,title
19489,True,Erotic Nights of the Living Dead
28701,True,Standoff
31934,True,Electrical Girl
32113,True,Diet of Sex
39901,True,Amateur Porn Star Killer 2
39902,True,The Band
40574,True,The Sinful Dwarf
41009,True,Adulterers
43090,True,Half -Life


In [7]:
movies[(movies['title'] == 'Amateur Porn Star Killer') | (movies['title'] == 'Amateur Porn Star Killer 3: The Final Chapter')][['adult', 'title']]

,adult,title
39899,False,Amateur Porn Star Killer
39900,False,Amateur Porn Star Killer 3: The Final Chapter


Feature does not seem to make much sense. There are only 9 movies with adult == True. The value is probably not correct, see the exaple Amateur Porn Star Killer 2 and Amateur Porn Star Killer, Amateur Porn Star Killer 3: The Final Chapter

In [8]:
del movies['adult']

### Removing budget

In [9]:
movies['budget'].value_counts()[:5]

0           36573
5000000       286
10000000      259
20000000      243
2000000       242
Name: budget, dtype: int64

In [10]:
# too many missing values 
del movies['budget']

### Removing revenue

In [11]:
movies['revenue'].value_counts().values[:5]

array([38052,    20,    19,    19,    18])

In [12]:
del movies['revenue']

## Removing Rows - Removing movies out of the dataset

### Removing broken data points

In [13]:
# broken ids
movies = movies[movies['id'] != '2012-09-29']
movies = movies[movies['id'] != '1997-08-20']
movies = movies[movies['id'] != '2014-01-01']

### Removing non existent or not yet released movies

In [14]:
movies['status'].value_counts()

Released           45014
Rumored              230
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [15]:
movies = movies[movies['status'] == 'Released']
del movies['status']

### Removing movies without posters

In [16]:
print('{} movies before removal'.format(movies.shape[0]))
movies_wo_posters = movies[movies['poster_path'].isnull()]
merge = pd.merge(movies, movies_wo_posters, how='outer', indicator=True)
movies = merge.loc[merge._merge == 'left_only']
del merge
del movies['_merge']
print('{} movies after removal'.format(movies.shape[0]))

45014 movies before removal
44641 movies after removal


## Transforming features

### Transforming genres to one hot

In [17]:
# read the weird json string
movies['genres'] = movies['genres'].apply(literal_eval)
# transform json dict to list of genres
genres = []
for movie in movies['genres']:
    mg = []
    for genre in movie:
        mg.append(genre['name'])
    genres.append(mg)

movies['genres'] = genres

In [18]:
def feature_one_hot(movie_dataset, column): 
    'Returns the given movie dataset with a transformed to one-hot encoded genre column.'

    # MultiLabelBinarizer is a very fast solution for one hot encoding on large dataframes
    mlb = MultiLabelBinarizer()
    one_hot_genre = pd.DataFrame(mlb.fit_transform(movie_dataset[column]),
                                 columns=mlb.classes_, 
                                 index=movie_dataset.index)

    movie_dataset = pd.concat([movie_dataset, one_hot_genre], sort=False, axis=1)
    movie_dataset.drop(column, axis=1)
    
    return movie_dataset

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# list of genres to one hot
movies = feature_one_hot(movies, 'genres')
del movies['genres']

In [20]:
movies.columns

Index(['id', 'imdb_id', 'original_language', 'overview', 'popularity',
       'poster_path', 'release_date', 'runtime', 'spoken_languages', 'tagline',
       'title', 'vote_average', 'vote_count', 'Action', 'Adventure',
       'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'],
      dtype='object')

In [21]:
genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
       'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War',
       'Western']
print('Amount of different Genres: ', len(genres))
for genre in genres:
    print('Genre:', genre, 'Occurrences:', sum(movies[genre]))

Amount of different Genres:  20
Genre: Action Occurrences: 6544
Genre: Adventure Occurrences: 3473
Genre: Animation Occurrences: 1909
Genre: Comedy Occurrences: 13040
Genre: Crime Occurrences: 4277
Genre: Documentary Occurrences: 3803
Genre: Drama Occurrences: 20024
Genre: Family Occurrences: 2739
Genre: Fantasy Occurrences: 2290
Genre: Foreign Occurrences: 1586
Genre: History Occurrences: 1389
Genre: Horror Occurrences: 4634
Genre: Music Occurrences: 1581
Genre: Mystery Occurrences: 2451
Genre: Romance Occurrences: 6655
Genre: Science Fiction Occurrences: 3015
Genre: TV Movie Occurrences: 752
Genre: Thriller Occurrences: 7559
Genre: War Occurrences: 1318
Genre: Western Occurrences: 1037


There is no genre which is rarely present except maybe TV Movie. Therefore, all genres will stay in the dataset and will not be deleted.

### Transforming original_language to one hot (only languages which are very frequent)

In [22]:
movies['original_language'].value_counts()

en    31641
fr     2409
it     1509
ja     1340
de     1063
es      982
ru      817
hi      501
ko      442
zh      406
sv      382
pt      313
cn      310
fi      277
nl      236
da      224
pl      209
tr      141
cs      128
el      111
no      105
fa       98
hu       97
ta       77
th       75
he       66
sr       63
ro       57
te       45
ar       38
      ...  
eu        3
kk        3
ky        3
kn        3
ps        2
bo        2
af        2
ne        2
iu        2
am        2
pa        2
lo        2
mn        2
lb        1
jv        1
fy        1
eo        1
uz        1
tg        1
qu        1
ay        1
mt        1
gl        1
rw        1
zu        1
si        1
hy        1
sm        1
la        1
cy        1
Name: original_language, Length: 89, dtype: int64

There are a lot of languages but only some are frequently present and only the ones that are frequent will be taken into account.

In [23]:
frequent_languages = movies['original_language'].value_counts().index[:7]
# initalize all languages with 0
for lang in frequent_languages:
    movies[lang] = 0
    
for index, row in movies.iterrows():
    if row['original_language'] in frequent_languages:
        movies.at[index, row['original_language']] = 1

del movies['original_language']

### Transforming release_date to release_year

The year of the release has to be enough information, the day or month are not of interest.

In [24]:
def isNaN(value):
    return value != value

In [25]:
from datetime import datetime
movies['release_year'] = [datetime.strptime(x, '%Y-%m-%d').year if not isNaN(x) else np.nan for x in movies['release_date'] ]
del movies['release_date']

## Adding new features from credits.csv

In [26]:
credits = pd.read_csv(os.path.join(data_path, 'credits.csv'))
credits['id'] = [int(x) for x in credits['id']]
movies['id'] = [int(x) for x in movies['id']]

### Adding features: star_1, star_2, star_3, director, writer, producer

In [27]:
def get_director(crew):
    for person in crew:
        if person['job'] == 'Director':
            return person['name']
    return np.nan

def get_writer(crew):
    for person in crew:
        if person['job'] == 'Writer':
            return person['name']
    return np.nan

def get_producer(crew):
    for person in crew:
        if person['job'] == 'Producer':
            return person['name']
    return np.nan

In [28]:
credits['cast'] = credits['cast'].apply(literal_eval) # read the weird json formated data
credits['cast'] = credits['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
credits['star_1'] = [x[0] if len(x) >= 1 else [] for x in credits['cast'] ]
credits['star_2'] = [x[1] if len(x) >= 2 else [] for x in credits['cast'] ]
credits['star_3'] = [x[2] if len(x) >= 3 else [] for x in credits['cast'] ]

credits['crew'] = credits['crew'].apply(literal_eval)
credits['director'] = credits['crew'].apply(get_director)
credits['writer'] = credits['crew'].apply(get_writer)
credits['producer'] = credits['crew'].apply(get_producer)

In [29]:
new_features = credits[['id', 'star_1', 'star_2', 'star_3', 'director', 'writer', 'producer']]
movies = movies.merge(new_features, on='id')

## Adding most frequent keywords from keywords.csv

In [30]:
keywords = pd.read_csv(os.path.join(data_path, 'keywords.csv'))

In [31]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [32]:
keywords['keywords'] = keywords['keywords'].apply(literal_eval)
keywords['keywords'] = keywords['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [33]:
keywords.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


In [35]:
keywords_one_hot = feature_one_hot(keywords, 'keywords')

In [36]:
keywords_one_hot.columns[2:]

Index([''comfort women'', '077', '10th century', '1500s', '15th birthday',
       '15th century', '16th century', '17th century', '18th century', '1910s',
       ...
       '부러진 화살', '소원', '알투비 : 리턴투베이스', '오싹한 연애', '위험한 소문', '찌라시',
       '찌라시 : 위험한 소문', '카운트다운', '하울링', '형사 duelist'],
      dtype='object', length=19956)

In [37]:
k, c = [], []
for keyword in keywords_one_hot.columns[2:]:
    k.append(keyword)
    c.append(sum(keywords_one_hot[keyword].values))

In [38]:
keyword_count = pd.DataFrame({'keyword': k, 'count': c})

In [39]:
keyword_count.sort_values('count', ascending=False, inplace=True)

In [40]:
# adding the 20 most frequent keywords to the dataset
keyword_count['keyword'][:20]

19592          woman director
8823         independent film
11651                  murder
1353           based on novel
11716                 musical
15806                     sex
19018                violence
12297                  nudity
1682                biography
14797                 revenge
17431                suspense
10360                    love
6371            female nudity
16803                   sport
13422                  police
17731                teenager
5313     duringcreditsstinger
15752                  sequel
6993               friendship
19684            world war ii
Name: keyword, dtype: object

In [41]:
most_frequent_keywords = keywords_one_hot[np.append(keyword_count['keyword'][:20].values, 'id')]

In [42]:
most_frequent_keywords

,woman director,independent film,murder,based on novel,musical,sex,violence,nudity,biography,revenge,...,love,female nudity,sport,police,teenager,duringcreditsstinger,sequel,friendship,world war ii,id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,862
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8844
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,15602
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,31357
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11862
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,949
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11860
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45325
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9091
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,710


In [43]:
movies = movies.merge(most_frequent_keywords, on='id')

## Exporting the preprocessed "The Movie Dataset" to csv

In [44]:
print('Preprocessing done!')

Preprocessing done!


In [45]:
movies.head()

,id,imdb_id,overview,popularity,poster_path,runtime,spoken_languages,tagline,title,vote_average,...,suspense,love,female nudity,sport,police,teenager,duringcreditsstinger,sequel,friendship,world war ii
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,...,0,0,0,0,0,0,0,0,1,0
1,8844,tt0113497,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,...,0,0,0,0,0,0,0,0,0,0
2,15602,tt0113228,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,...,0,0,0,0,0,0,1,0,0,0
3,31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,...,0,0,0,0,0,0,0,0,0,0
4,11862,tt0113041,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,...,0,0,0,0,0,0,0,0,0,0


In [46]:
movies.to_csv('movies_preprocessed.csv', index=False)